# FindYourLandlord Data Cleaning
### This project aims to:
* provide a visual representation for who or what entity owns the most property, what other properties the owner owns, and where
* provide tenants in North Jersey with pertinent information about who or what owns their building or apartment
### Beginning with Jersey City, I'll eventually expand to other cities throughout North Jersey: Paterson, Newark, Hackensack…
### This notebook is the data portion of the project, where I will test out methods for cleaning and extracting the relevant data. The data will eventually be picked up by React and mapboxGL.
### The csv's were retrieved from Monmouth County's Tax Assessor website:
### https://tax1.co.monmouth.nj.us/cgi-bin/prc6.cgi?menu=index&ms_user=monm&passwd=data&district=1301&mode=11

## To Do:
* the data is unbelievably irregular: needs cleaning and normalization
* create column of integers representing the number of unique properties associated with the owner: this informs the size of the circle's radius on the map
* create column of an array of strings of the properties associated with the owner: this informs the size of the bounding box of selected properties on the map
* units: if the cells in the column of string arrays has a len of 1, but the column of integers is < 1, it represents the number of units at the property

In [2]:
import pandas as pd
import os
import numpy as np
from geopy.geocoders import Nominatim
from geopy.extra.rate_limiter import RateLimiter

In [3]:
# for returning lat long coordinates from addresses
geolocator = Nominatim(user_agent="myGeolocator", timeout=2)

In [4]:
hackensack = pd.read_csv('0223monm205221.csv', dtype={14: str, 20: str, 24: str})

jersey_city = pd.read_csv('0906monm220028.csv', dtype={20: str,
                                                       22: str,
                                                       23: str,
                                                       25: str,
                                                       42: str,
                                                       43: str,
                                                       47: str,
                                                       87: str,
                                                       89: str})

jersey_city.rename(columns={'Property Location': 'property_location', 
                            "Owner's Name": 'owners_name', 
                            "Owner's Mailing Address": 'owners_mailing_address', 
                            "City/State/Zip": "city_state_zip"}, inplace=True)

hackensack.rename(columns={'Property Location': 'property_location', 
                            "Owner's Name": 'owners_name', 
                            "Owner's Mailing Address": 'owners_mailing_address',
                            "City/State/Zip": "city_state_zip"}, inplace=True)

# title() case is necessary in order to return coordinates with Nominatim 
# we also can't assume the same city has the same zipcode throughout
# the zipcode must be extracted from the city/state/zip column
jersey_city['property_full_address'] = jersey_city.property_location.str.title() + " " + jersey_city.city_state_zip.str.title()

In [5]:
jersey_city.columns

Index(['Municipality', 'Block', 'Lot', 'Qual', 'property_location',
       'Property Class', 'owners_name', 'owners_mailing_address',
       'city_state_zip', 'Sq. Ft.', 'Yr. Built', 'Building Class',
       'Prior Block', 'Prior Lot', 'Prior Qual', 'Updated', 'Zone', 'Account',
       'Mortgage Account', 'Bank Code', 'Sp Tax Cd', 'Sp Tax Cd.1',
       'Sp Tax Cd.2', 'Sp Tax Cd.3', 'Map Page', 'Additional Lots',
       'Land Desc', 'Building Desc', 'Class 4 Code', 'Acreage', 'EPL Own',
       'EPL Use', 'EPL Desc', 'EPL Statute', 'EPL Init', 'EPL Further',
       'EPL Facility Name', 'Taxes 1', 'Taxes 2', 'Taxes 3', 'Taxes 4',
       'Sale Date', 'Deed Book', 'Deed Page', 'Sale Price', 'NU Code', 'Ratio',
       'Type/Use', 'Year', 'Owner', 'Street', 'City/State/Zip.1',
       'Land Assmnt', 'Building Assmnt', 'Exempt', 'Total Assmnt', 'Assessed',
       'Year.1', 'Owner.1', 'Street.1', 'City/State/Zip.2', 'Land Assmnt.1',
       'Building Assmnt.1', 'Exempt.1', 'Total Assmnt.1', 'Asse

#### The following five cells are columns which can be deleted from the dataframe...
#### ...I needed to figure out their respective indices

In [6]:
jersey_city.columns[5]

'Property Class'

In [7]:
jersey_city.columns[9]

'Sq. Ft.'

In [8]:
jersey_city.columns[11:83]

Index(['Building Class', 'Prior Block', 'Prior Lot', 'Prior Qual', 'Updated',
       'Zone', 'Account', 'Mortgage Account', 'Bank Code', 'Sp Tax Cd',
       'Sp Tax Cd.1', 'Sp Tax Cd.2', 'Sp Tax Cd.3', 'Map Page',
       'Additional Lots', 'Land Desc', 'Building Desc', 'Class 4 Code',
       'Acreage', 'EPL Own', 'EPL Use', 'EPL Desc', 'EPL Statute', 'EPL Init',
       'EPL Further', 'EPL Facility Name', 'Taxes 1', 'Taxes 2', 'Taxes 3',
       'Taxes 4', 'Sale Date', 'Deed Book', 'Deed Page', 'Sale Price',
       'NU Code', 'Ratio', 'Type/Use', 'Year', 'Owner', 'Street',
       'City/State/Zip.1', 'Land Assmnt', 'Building Assmnt', 'Exempt',
       'Total Assmnt', 'Assessed', 'Year.1', 'Owner.1', 'Street.1',
       'City/State/Zip.2', 'Land Assmnt.1', 'Building Assmnt.1', 'Exempt.1',
       'Total Assmnt.1', 'Assessed.1', 'Year.2', 'Owner.2', 'Street.2',
       'City/State/Zip.3', 'Land Assmnt.2', 'Building Assmnt.2', 'Exempt.2',
       'Total Assmnt.2', 'Assessed.2', 'Year.3', 'Owner.3

In [9]:
jersey_city.columns[83]

'Assessed.3'

In [10]:
jersey_city.columns[86:91]

Index(['Neigh', 'VCS', 'StyDesc', 'Style', 'Unnamed: 90'], dtype='object')

## Does the owner live in the building? 
## Does the 'property location' match the 'owner's mailing address'?

In [11]:
# this method is inefficient, given we are searching 60k+ rows
jersey_city.property_location == jersey_city.owners_mailing_address

0        False
1        False
2        False
3        False
4        False
         ...  
63829    False
63830    False
63831    False
63832    False
63833    False
Length: 63834, dtype: bool

In [12]:
# numpy has better methods for quickly checking
# a quick look by wrapping the same equivalence in a numpy function to see if our intuition is correct
np.where(jersey_city.property_location == jersey_city.owners_mailing_address)

(array([   27,    29,    34, ..., 63776, 63796, 63797]),)

### property location matching the owners mailing address…can we safely assume they reside at the address?
### as many as 16544 owners reside in the building they own: 
### however they might not all be residential properties

In [13]:
jersey_city.loc[jersey_city.property_location == jersey_city.owners_mailing_address]

,Municipality,Block,Lot,Qual,property_location,Property Class,owners_name,owners_mailing_address,city_state_zip,Sq. Ft.,...,Total Assmnt.3,Assessed.3,Latitude,Longitude,Neigh,VCS,StyDesc,Style,Unnamed: 90,property_full_address
27,906,201.0,7.0,NaN,817 TONNELE AVE.,1,"NELMIR REALTY, L.L.C.",817 TONNELE AVE.,"JERSEY CITY, NJ 07307",0,...,111900,111900.0,0,0,NaN,NaN,NaN,NaN,NaN,"817 Tonnele Ave. Jersey City, Nj 07307"
29,906,201.0,9.0,NaN,823 TONNELE AVE.,4A,GERRY GAS SUPPLY INC.C/O LILH,823 TONNELE AVE.,"JERSEY CITY, N.J. 07307",0,...,1699500,1699500.0,0,0,NaN,NaN,NaN,NaN,NaN,"823 Tonnele Ave. Jersey City, N.J. 07307"
34,906,301.0,1.0,NaN,677 LIBERTY AVE.,2,"PEDDI, PRADEEP",677 LIBERTY AVE.,"JERSEY CITY, N.J. 07307",1600,...,416500,416500.0,0,0,NaN,NaN,NaN,NaN,NaN,"677 Liberty Ave. Jersey City, N.J. 07307"
35,906,301.0,2.0,NaN,675 LIBERTY AVE.,2,"PAREJA, HENRY A. & MIRYAM C.",675 LIBERTY AVE.,"JERSEY CITY, N.J. 07307",1616,...,363300,363300.0,0,0,NaN,NaN,NaN,NaN,NaN,"675 Liberty Ave. Jersey City, N.J. 07307"
36,906,301.0,3.0,NaN,673 LIBERTY AVE.,2,"HIRPARA, PRAVIN",673 LIBERTY AVE.,"JERSEY CITY, NJ 07307",1600,...,364900,364900.0,0,0,NaN,NaN,NaN,NaN,NaN,"673 Liberty Ave. Jersey City, Nj 07307"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
63740,906,30304.0,14.0,NaN,100 GARFIELD AVE.,2,"SERVIDA, AUREA",100 GARFIELD AVE.,"JERSEY CITY, NJ 07305",1882,...,271000,271000.0,0,0,NaN,NaN,NaN,NaN,NaN,"100 Garfield Ave. Jersey City, Nj 07305"
63742,906,30304.0,16.0,NaN,104 GARFIELD AVE.,2,"RODRIGUEZ, BENJAMIN",104 GARFIELD AVE.,"JERSEY CITY, N.J. 07305",2412,...,268200,268200.0,0,0,NaN,NaN,NaN,NaN,NaN,"104 Garfield Ave. Jersey City, N.J. 07305"
63776,906,30306.0,8.0,NaN,100 SUMMIT PLACE,4B,"SUMMIT/GREENWICH URBAN RENEWAL, LLC",100 SUMMIT PLACE,"JERSEY CITY, NJ 07305",0,...,13139700,13139700.0,0,0,NaN,NaN,NaN,NaN,NaN,"100 Summit Place Jersey City, Nj 07305"
63796,906,30307.0,10.0,NaN,109 PORT JERSEY BLVD.,4B,DOMISA LLC,109 PORT JERSEY BLVD.,"JERSEY CITY, N.J. 07305",0,...,2863800,2863800.0,0,0,NaN,NaN,NaN,NaN,NaN,"109 Port Jersey Blvd. Jersey City, N.J. 07305"


## Does the property owner own more than one property? Which property owner owns the most properties?
### These numbers can be deceiving; many properties or units can exist in a single address.
### The same address can be included twice. 

In [14]:
# As we'll see below, many of the addresses and owners are spelled incorrectly 
len(jersey_city['owners_name'].unique())

49247

In [15]:
len(jersey_city['property_location'].unique())

37842

### multiple units can appear in a single property, adding to the value counts below

## How many LLCs have "UR" or "Urban Renewal" in the owner's name?

In [17]:
# More examples of how unclean the data is, even on a state database
jersey_city[jersey_city.owners_name.str.contains('URBAN RENEWAL')]['owners_name'].unique()

array(['HEIGHTS URBAN RENEWAL SENIOR H.',
       'RATAN JERSEY CITY URBAN RENEWAL,LLC',
       'BRASS WORKS URBAN RENEWAL CO., LLC',
       'HUDSON PALISADES URBAN RENEWAL,LLC',
       '364 NINTH STREET URBAN RENEWAL, LLC',
       'NINTH STREET TWO URBAN RENEWAL,LLC',
       'TOWER EAST URBAN RENEWAL COMPANY',
       'JAMES MONROE URBAN RENEWAL CO.', '25 RIVER DR.SO.URBAN RENEWAL',
       'SENATE PLACE URBAN RENEWAL, LLC',
       'H.P. LINCOLN URBAN RENEWAL COMPANY',
       'ERIE 10TH URBAN RENEWAL, L.L.C.', '9TH STREET URBAN RENEWAL,LLC',
       'VAN WAGENEN II URBAN RENEWAL, LLC.',
       '17-19 DIVISION ST URBAN RENEWAL,LLC',
       '380 NEWARK REALTY URBAN RENEWAL LLC',
       'BLOCK 284 NORTH URBAN RENEWAL, LLC',
       '500 MANILA AVE.URBAN RENEWAL, LLC',
       '500 MANILA AVE.URBAN RENEWAL LLC',
       'KRE HAMILTON URBAN RENEWAL LLC',
       'VAN WAGENEN I URBAN RENEWAL, LLC.',
       '160 FIRST STREET URBAN RENEWAL, LLC',
       'PS FIRST HUDSON URBAN RENEWAL LLC',
       '14

In [18]:
jersey_city[jersey_city.owners_name.str.contains(' UR ')]['owners_name'].unique()

array(['SUMMIT PLAZA ASSOCIATES, UR LTD PTN',
       'PADUA COURT UR C/O INSPIRED VISION',
       'PADUA COURT UR C/O INSPRED VISION',
       'CAL-HARBOR SO. PIER UR ASC %M CALI'], dtype=object)

In [19]:
jersey_city[jersey_city.owners_name == 'COA 99 HUDSON,LLC']['property_location'].unique()

array(['99 HUDSON ST.'], dtype=object)

In [154]:
test_data.columns

Index(['property_location', 'owners_name', 'owners_mailing_address',
       'city_state_zip', 'Yr. Built', 'Latitude', 'Longitude',
       'property_full_address', 'gcode'],
      dtype='object')

In [20]:
def return_jc_properties(owner=str): 
    return jersey_city[jersey_city.owners_name == owner.rstrip()]['property_location'].unique()

In [152]:
def return_testData_properties(owner=str): 
    return len(test_data[test_data.owners_name == owner.rstrip()]['property_location'].unique())

In [160]:
def return_owner(address=str):
    owner = test_data[test_data.property_location == address.strip()]['owners_name'] 
    return owner
    # if the following conditional is true, it means the owner is associated with other properties 
    if len(owner) < 1:
        other_properties = test_data[test_data.owners_name == owner]['property_location'].unique()
        # return the other address or addresses associated with owner
        return other_properties

In [175]:
def return_properties(owner=str):
    unique_properties = test_data[test_data.owners_name == owner]['property_location'].unique()
    units = len(test_data[test_data.owners_name == owner]['property_location'])
    if units > 1:
        return f'{units} units at {unique_properties[0]}'
    else:
        return unique_properties

In [176]:
return_properties('HUDSON MAIN URBAN RENEWAL,LLC')

'2 units at 143 CHAPEL AVE'

In [168]:
return_owner('143 CHAPEL AVE	')

632    HUDSON MAIN URBAN RENEWAL,LLC
633    HUDSON MAIN URBAN RENEWAL,LLC
Name: owners_name, dtype: object

In [153]:
return_testData_properties('HOUSING AUTHORITY OF JERSEY CITY')

45

In [22]:
%%time
# a smaller dataset to view inconsistencies
# are there duplicate addresses in this subset of data?
# are the addresses accurate? 

# a copy() is used to avoid writing on the original dataframe
js_ha = jersey_city[jersey_city.owners_name == 'HOUSING AUTHORITY OF JERSEY CITY'].copy()
js_ha.reset_index(inplace=True, drop=True)
js_ha['gcode'] = js_ha.property_full_address.apply(geolocator.geocode)

city_of_js = jersey_city[jersey_city.owners_name == 'CITY OF JERSEY CITY'].copy()
city_of_js.reset_index(inplace=True, drop=True)
city_of_js['gcode'] = city_of_js.property_full_address.apply(geolocator.geocode)

urban_renewal = jersey_city[jersey_city.owners_name.str.contains('URBAN RENEWAL')].copy()
urban_renewal.reset_index(inplace=True, drop=True)
urban_renewal['gcode'] = urban_renewal.property_full_address.apply(geolocator.geocode)

CPU times: user 1.46 s, sys: 224 ms, total: 1.69 s
Wall time: 7min 11s


In [25]:
# for now, we'll drop the None values in the gcode column to make our lives easier 
# this does not however solve the issue of duplicate values in that column
js_ha.dropna(axis=0, subset=['gcode'], inplace=True)
city_of_js.dropna(axis=0, subset=['gcode'], inplace=True)
urban_renewal.dropna(axis=0, subset=['gcode'], inplace=True)

# since the dataframe already contains Latitude and Longitude columns, we can write to them directly
js_ha['Latitude'] = [g.latitude for g in js_ha.gcode]
js_ha['Longitude'] = [g.longitude for g in js_ha.gcode]

city_of_js['Latitude'] = [g.latitude for g in city_of_js.gcode]
city_of_js['Longitude'] = [g.longitude for g in city_of_js.gcode]

urban_renewal['Latitude'] = [g.latitude for g in urban_renewal.gcode]
urban_renewal['Longitude'] = [g.longitude for g in urban_renewal.gcode]

In [26]:
# a less tedious way to execute this might be drop ranges of columns instead of doing it all manually
js_ha.drop(columns=['Municipality', 
                    'Block', 
                    'Lot', 
                    'Qual', 
                    'Property Class', 
                    'Sq. Ft.', 
                    'Building Class', 
                    'Prior Block', 
                    'Prior Lot', 
                    'Prior Qual', 
                    'Updated', 
                    'Zone', 
                    'Account', 
                    'Mortgage Account', 
                    'Bank Code', 
                    'Sp Tax Cd', 
                    'Sp Tax Cd.1', 
                    'Sp Tax Cd.2', 
                    'Sp Tax Cd.3', 
                    'Map Page',
                    'Additional Lots', 
                    'Land Desc', 
                    'Building Desc', 
                    'Class 4 Code', 
                    'Acreage', 
                    'EPL Own', 
                    'EPL Use', 
                    'EPL Desc', 
                    'EPL Statute', 
                    'EPL Init', 
                    'EPL Further', 
                    'EPL Facility Name', 
                    'Taxes 1', 
                    'Taxes 2', 
                    'Taxes 3', 
                    'Taxes 4', 
                    'Sale Date', 
                    'Deed Book', 
                    'Deed Page', 
                    'Sale Price', 
                    'NU Code', 
                    'Ratio', 
                    'Type/Use', 
                    'Year', 
                    'Owner', 
                    'Street', 
                    'City/State/Zip.1',
                    'Land Assmnt', 
                    'Building Assmnt', 
                    'Exempt', 
                    'Total Assmnt', 
                    'Assessed',
                    'Year.1', 
                    'Owner.1', 
                    'Street.1', 
                    'City/State/Zip.2', 
                    'Land Assmnt.1',
                    'Building Assmnt.1', 
                    'Exempt.1', 
                    'Total Assmnt.1', 
                    'Assessed.1',
                    'Year.2', 
                    'Owner.2', 
                    'Street.2', 
                    'City/State/Zip.3', 
                    'Land Assmnt.2',
                    'Building Assmnt.2', 
                    'Exempt.2', 
                    'Total Assmnt.2', 
                    'Assessed.2',
                    'Year.3', 
                    'Owner.3', 
                    'Street.3', 
                    'City/State/Zip.4', 
                    'Land Assmnt.3',
                    'Building Assmnt.3', 
                    'Exempt.3', 
                    'Total Assmnt.3', 
                    'Assessed.3', 
                    'Neigh', 
                    'VCS', 
                    'StyDesc', 
                    'Style',
                    'Unnamed: 90'], inplace=True) 

In [27]:
# a less tedious way to execute this might be drop ranges of columns instead of doing it all manually
city_of_js.drop(columns=['Municipality', 
                    'Block', 
                    'Lot', 
                    'Qual', 
                    'Property Class', 
                    'Sq. Ft.', 
                    'Building Class', 
                    'Prior Block', 
                    'Prior Lot', 
                    'Prior Qual', 
                    'Updated', 
                    'Zone', 
                    'Account', 
                    'Mortgage Account', 
                    'Bank Code', 
                    'Sp Tax Cd', 
                    'Sp Tax Cd.1', 
                    'Sp Tax Cd.2', 
                    'Sp Tax Cd.3', 
                    'Map Page',
                    'Additional Lots', 
                    'Land Desc', 
                    'Building Desc', 
                    'Class 4 Code', 
                    'Acreage', 
                    'EPL Own', 
                    'EPL Use', 
                    'EPL Desc', 
                    'EPL Statute', 
                    'EPL Init', 
                    'EPL Further', 
                    'EPL Facility Name', 
                    'Taxes 1', 
                    'Taxes 2', 
                    'Taxes 3', 
                    'Taxes 4', 
                    'Sale Date', 
                    'Deed Book', 
                    'Deed Page', 
                    'Sale Price', 
                    'NU Code', 
                    'Ratio', 
                    'Type/Use', 
                    'Year', 
                    'Owner', 
                    'Street', 
                    'City/State/Zip.1',
                    'Land Assmnt', 
                    'Building Assmnt', 
                    'Exempt', 
                    'Total Assmnt', 
                    'Assessed',
                    'Year.1', 
                    'Owner.1', 
                    'Street.1', 
                    'City/State/Zip.2', 
                    'Land Assmnt.1',
                    'Building Assmnt.1', 
                    'Exempt.1', 
                    'Total Assmnt.1', 
                    'Assessed.1',
                    'Year.2', 
                    'Owner.2', 
                    'Street.2', 
                    'City/State/Zip.3', 
                    'Land Assmnt.2',
                    'Building Assmnt.2', 
                    'Exempt.2', 
                    'Total Assmnt.2', 
                    'Assessed.2',
                    'Year.3', 
                    'Owner.3', 
                    'Street.3', 
                    'City/State/Zip.4', 
                    'Land Assmnt.3',
                    'Building Assmnt.3', 
                    'Exempt.3', 
                    'Total Assmnt.3', 
                    'Assessed.3', 
                    'Neigh', 
                    'VCS', 
                    'StyDesc', 
                    'Style',
                    'Unnamed: 90'], inplace=True) 

In [28]:
urban_renewal.drop(columns=['Municipality', 
                    'Block', 
                    'Lot', 
                    'Qual', 
                    'Property Class', 
                    'Sq. Ft.', 
                    'Building Class', 
                    'Prior Block', 
                    'Prior Lot', 
                    'Prior Qual', 
                    'Updated', 
                    'Zone', 
                    'Account', 
                    'Mortgage Account', 
                    'Bank Code', 
                    'Sp Tax Cd', 
                    'Sp Tax Cd.1', 
                    'Sp Tax Cd.2', 
                    'Sp Tax Cd.3', 
                    'Map Page',
                    'Additional Lots', 
                    'Land Desc', 
                    'Building Desc', 
                    'Class 4 Code', 
                    'Acreage', 
                    'EPL Own', 
                    'EPL Use', 
                    'EPL Desc', 
                    'EPL Statute', 
                    'EPL Init', 
                    'EPL Further', 
                    'EPL Facility Name', 
                    'Taxes 1', 
                    'Taxes 2', 
                    'Taxes 3', 
                    'Taxes 4', 
                    'Sale Date', 
                    'Deed Book', 
                    'Deed Page', 
                    'Sale Price', 
                    'NU Code', 
                    'Ratio', 
                    'Type/Use', 
                    'Year', 
                    'Owner', 
                    'Street', 
                    'City/State/Zip.1',
                    'Land Assmnt', 
                    'Building Assmnt', 
                    'Exempt', 
                    'Total Assmnt', 
                    'Assessed',
                    'Year.1', 
                    'Owner.1', 
                    'Street.1', 
                    'City/State/Zip.2', 
                    'Land Assmnt.1',
                    'Building Assmnt.1', 
                    'Exempt.1', 
                    'Total Assmnt.1', 
                    'Assessed.1',
                    'Year.2', 
                    'Owner.2', 
                    'Street.2', 
                    'City/State/Zip.3', 
                    'Land Assmnt.2',
                    'Building Assmnt.2', 
                    'Exempt.2', 
                    'Total Assmnt.2', 
                    'Assessed.2',
                    'Year.3', 
                    'Owner.3', 
                    'Street.3', 
                    'City/State/Zip.4', 
                    'Land Assmnt.3',
                    'Building Assmnt.3', 
                    'Exempt.3', 
                    'Total Assmnt.3', 
                    'Assessed.3', 
                    'Neigh', 
                    'VCS', 
                    'StyDesc', 
                    'Style',
                    'Unnamed: 90'], inplace=True) 

In [120]:
test_data = js_ha.append([city_of_js, urban_renewal])
test_data.reset_index(inplace=True, drop=True)

In [121]:
test_data

,property_location,owners_name,owners_mailing_address,city_state_zip,Yr. Built,Latitude,Longitude,property_full_address,gcode
0,514 NEWARK AVE.,HOUSING AUTHORITY OF JERSEY CITY,514 NEWARK AVENUE,"JERSEY CITY, NJ 07306",NaN,40.730082,-74.055283,"514 Newark Ave. Jersey City, Nj 07306","(514, Newark Avenue, Jersey City, Hudson Count..."
1,88 ERIE ST.,HOUSING AUTHORITY OF JERSEY CITY,400 U.S. HIGHWAY #1,"JERSEY CITY, NJ 07306",1870.0,40.724742,-74.043713,"88 Erie St. Jersey City, Nj 07306","(88, Erie Street, Newport, Jersey City, Hudson..."
2,57 DALES AVE.,HOUSING AUTHORITY OF JERSEY CITY,400 ROUTE 1,"JERSEY CITY, NJ 07306",1941.0,40.735895,-74.076336,"57 Dales Ave. Jersey City, Nj 07306","(57, Dales Avenue, Marion, Jersey City, Hudson..."
3,547 MONTGOMERY ST.,HOUSING AUTHORITY OF JERSEY CITY,400 U.S. HIGHWAY #1,"JERSEY CITY, NJ 07306",NaN,40.722429,-74.060221,"547 Montgomery St. Jersey City, Nj 07306","(547, Montgomery Street, Bergen, Jersey City, ..."
4,20 FLORENCE ST.,HOUSING AUTHORITY OF JERSEY CITY,400 U.S. HIGHWAY #1,"JERSEY CITY, NJ 07306",NaN,40.723183,-74.060375,"20 Florence St. Jersey City, Nj 07306","(Florence Street, Bergen, Jersey City, Hudson ..."
...,...,...,...,...,...,...,...,...,...
631,455 OCEAN AVE.,GENESIS OCEAN URBAN RENEWAL ASSOC.,455 OCEAN AVE.,"JERSEY CITY , NEW JERSEY 07305",2018.0,40.702127,-74.082018,"455 Ocean Ave. Jersey City , New Jersey 07305","(455, Ocean Avenue, Greenville, Jersey City, H..."
632,143 CHAPEL AVE,"HUDSON MAIN URBAN RENEWAL,LLC",17 INDEPENDENCE WAY,"JERSEY CITY, NJ 07302",NaN,40.694191,-74.084210,"143 Chapel Ave Jersey City, Nj 07302","(143, Chapel Avenue, Jersey City, Hudson Count..."
633,143 CHAPEL AVE,"HUDSON MAIN URBAN RENEWAL,LLC",17 INDEPENDENCE WAY,"JERSEY CITY, NJ 07302",2017.0,40.694191,-74.084210,"143 Chapel Ave Jersey City, Nj 07302","(143, Chapel Avenue, Jersey City, Hudson Count..."
634,100 SUMMIT PLACE,"SUMMIT/GREENWICH URBAN RENEWAL, LLC",100 SUMMIT PLACE,"JERSEY CITY, NJ 07305",2000.0,40.683216,-74.093701,"100 Summit Place Jersey City, Nj 07305","(100, Summit Place, Jersey City, Hudson County..."


In [143]:
# this groupby result will be used for the associatedProperties feature
# ie, the number of properties associated with a selected owner
# it will become the associatedProperties column

# cell 15 performed a similar function, however, this is more accurate


# number of properties associated with owners_name
test_data.groupby('owners_name')['property_location'].unique()

owners_name
140 BAY STREET URBAN RENEWAL, LLC                                          [140 BAY ST.]
160 FIRST STREET URBAN RENEWAL, LLC                                      [160 FIRST ST.]
170 LAFAYETTE URBAN RENEWAL,LLC                                      [170 LAFAYETTE ST.]
272 GROVE STREET URBAN RENEWAL, LLC                                 [201 MONTGOMERY ST.]
280 FAIRMOUNT URBAN RENEWAL, LLC.                                   [280 FAIRMOUNT AVE.]
364 NINTH STREET URBAN RENEWAL, LLC                                  [255 BRUNSWICK ST.]
380 NEWARK REALTY URBAN RENEWAL LLC                                    [380 NEWARK AVE.]
400 CLAREMENT URBAN RENEWAL LLC                                     [382 CLAREMONT AVE.]
400 CLAREMONT URBAN RENEWAL LLC                                     [400 CLAREMONT AVE.]
456 GRAND REALTY URBAN RENEWAL, LLC                                      [460 GRAND ST.]
70 COLUMBUS URBAN RENEWAL, LLC                                         [70 COLUMBUS DR.]
95 VAN DA

In [127]:
#addresses of the properties associated with the owners_name
[len(i) for i in test_data.groupby(test_data.owners_name)['property_location'].unique()]

In [134]:
test_data.groupby(test_data.owners_name, as_index=False)['property_location'].count()

,owners_name,property_location
0,"140 BAY STREET URBAN RENEWAL, LLC",1
1,"160 FIRST STREET URBAN RENEWAL, LLC",104
2,"170 LAFAYETTE URBAN RENEWAL,LLC",2
3,"272 GROVE STREET URBAN RENEWAL, LLC",1
4,"280 FAIRMOUNT URBAN RENEWAL, LLC.",2
5,"364 NINTH STREET URBAN RENEWAL, LLC",2
6,380 NEWARK REALTY URBAN RENEWAL LLC,5
7,400 CLAREMENT URBAN RENEWAL LLC,1
8,400 CLAREMONT URBAN RENEWAL LLC,1
9,"456 GRAND REALTY URBAN RENEWAL, LLC",1


In [122]:
jersey_city.groupby('owners_name', as_index=False)['property_location'].count()

,owners_name,property_location
0,"0LALEKAN, RODA",1
1,1 CAVEN POINT ROAD ASSOC. L.L.C.,1
2,1 EDWARD HART ROAD LLC,1
3,"1 EXCHANGE JC, LLC % RYAN, LLC",2
4,1-12 CATHERINE COURT JC LLC,1
...,...,...
49242,"ZYSKOWSKA, ANNA",1
49243,"ZYZYCK, STACEY",1
49244,"\OA 99 HUDSON,LLC",1
49245,"`ENG, MENG RAN",1


In [61]:
test_data.to_csv('test_data.csv', index=None)

### a list comprehension is used to return the raw data of the address
### a nested enumerating list comprehension is used to return the indices of unclean addresses
#### the example below is actually a list comprehension within a list comprehension 

In [22]:
[i for i,e in enumerate([geolocator.geocode(address) for address in js_ha.property_full_address]) if e == None]

[0, 4, 14, 15, 23, 24, 25, 26, 29, 30, 35, 36, 37, 39, 45, 54]

### We know the addresses are unclean when we substitute parts of the address
### For example, "Sixteenth" for "16th": 

In [162]:
geolocator.geocode('235 16th St. Jersey City, NJ 07307')

Location(235, 16th Street, Jersey City, Hudson County, New Jersey, 07310, United States, (40.73314597560976, -74.04304924390243, 0.0))

In [168]:
js_ha.property_full_address[23]

'9-31 Wilmot Ave. Jersey City, NJ 07307'

In [123]:
geolocator.geocode('61 Merritt St. Jersey City, NJ 07307').point

Point(40.6888175, -74.0973369, 0.0)

In [118]:
jersey_city[jersey_city.owners_name == '160 FIRST STREET URBAN RENEWAL, LLC']['property_location']

23540    160 FIRST ST.
23541    160 FIRST ST.
23544    160 FIRST ST.
23545    160 FIRST ST.
23556    160 FIRST ST.
             ...      
23857    160 FIRST ST.
23858    160 FIRST ST.
23859    160 FIRST ST.
23860    160 FIRST ST.
23861    160 FIRST ST.
Name: property_location, Length: 104, dtype: object

In [14]:
hackensack.owners_name.value_counts().head(50)

CITY OF HACKENSACK                     69
WORLD PLAZA AC, LLC                    29
ESSEX COURT REALTY CORP.               28
HACKENSACK DEVELOPERS, LLC             25
TERRACE SQUARE CONDOS % J LOMBARDO     21
N Y S & W R R C/O N. STECKLER          19
HACK & N.Y. R.R. CO C/O LAND & TAX     19
SKYVIEW AT HACKENSACK, LLC             19
COUNTY OF BERGEN                       16
WORLD PLAZA PROPERTIES, LLC            13
ATTESSA PROPERTIES LLC                 13
HEKEMIAN,SAMUEL %THE S HEKEMIAN GRP    12
BD OF ED CITY OF HACK                  12
ELYASH, ADELAIDA                       10
FAIRLEIGH DICKINSON UNIVERSITY          9
400 E MAIN STREET, LLC                  9
NEW HOPE BAPTIST CHURCH                 8
RUDDOCK, WAYNE & ROSA                   7
DI CAROLIS REALTY CO.                   7
JACKSON PINK LLC                        7
NESS REALTY 1 LLC                       7
MAIN PORTFOLIO LLC                      6
US BANK TRUST NA TRSTE                  6
CYPRUS HOLDING COMPANY LLC        

In [124]:
test_data

,property_location,owners_name,owners_mailing_address,city_state_zip,Yr. Built,Latitude,Longitude,property_full_address,gcode
0,514 NEWARK AVE.,HOUSING AUTHORITY OF JERSEY CITY,514 NEWARK AVENUE,"JERSEY CITY, NJ 07306",NaN,40.730082,-74.055283,"514 Newark Ave. Jersey City, Nj 07306","(514, Newark Avenue, Jersey City, Hudson Count..."
1,88 ERIE ST.,HOUSING AUTHORITY OF JERSEY CITY,400 U.S. HIGHWAY #1,"JERSEY CITY, NJ 07306",1870.0,40.724742,-74.043713,"88 Erie St. Jersey City, Nj 07306","(88, Erie Street, Newport, Jersey City, Hudson..."
2,57 DALES AVE.,HOUSING AUTHORITY OF JERSEY CITY,400 ROUTE 1,"JERSEY CITY, NJ 07306",1941.0,40.735895,-74.076336,"57 Dales Ave. Jersey City, Nj 07306","(57, Dales Avenue, Marion, Jersey City, Hudson..."
3,547 MONTGOMERY ST.,HOUSING AUTHORITY OF JERSEY CITY,400 U.S. HIGHWAY #1,"JERSEY CITY, NJ 07306",NaN,40.722429,-74.060221,"547 Montgomery St. Jersey City, Nj 07306","(547, Montgomery Street, Bergen, Jersey City, ..."
4,20 FLORENCE ST.,HOUSING AUTHORITY OF JERSEY CITY,400 U.S. HIGHWAY #1,"JERSEY CITY, NJ 07306",NaN,40.723183,-74.060375,"20 Florence St. Jersey City, Nj 07306","(Florence Street, Bergen, Jersey City, Hudson ..."
...,...,...,...,...,...,...,...,...,...
631,455 OCEAN AVE.,GENESIS OCEAN URBAN RENEWAL ASSOC.,455 OCEAN AVE.,"JERSEY CITY , NEW JERSEY 07305",2018.0,40.702127,-74.082018,"455 Ocean Ave. Jersey City , New Jersey 07305","(455, Ocean Avenue, Greenville, Jersey City, H..."
632,143 CHAPEL AVE,"HUDSON MAIN URBAN RENEWAL,LLC",17 INDEPENDENCE WAY,"JERSEY CITY, NJ 07302",NaN,40.694191,-74.084210,"143 Chapel Ave Jersey City, Nj 07302","(143, Chapel Avenue, Jersey City, Hudson Count..."
633,143 CHAPEL AVE,"HUDSON MAIN URBAN RENEWAL,LLC",17 INDEPENDENCE WAY,"JERSEY CITY, NJ 07302",2017.0,40.694191,-74.084210,"143 Chapel Ave Jersey City, Nj 07302","(143, Chapel Avenue, Jersey City, Hudson Count..."
634,100 SUMMIT PLACE,"SUMMIT/GREENWICH URBAN RENEWAL, LLC",100 SUMMIT PLACE,"JERSEY CITY, NJ 07305",2000.0,40.683216,-74.093701,"100 Summit Place Jersey City, Nj 07305","(100, Summit Place, Jersey City, Hudson County..."


In [125]:
i for i in test_data.owners_name

0         HOUSING AUTHORITY OF JERSEY CITY
1         HOUSING AUTHORITY OF JERSEY CITY
2         HOUSING AUTHORITY OF JERSEY CITY
3         HOUSING AUTHORITY OF JERSEY CITY
4         HOUSING AUTHORITY OF JERSEY CITY
                      ...                 
631     GENESIS OCEAN URBAN RENEWAL ASSOC.
632          HUDSON MAIN URBAN RENEWAL,LLC
633          HUDSON MAIN URBAN RENEWAL,LLC
634    SUMMIT/GREENWICH URBAN RENEWAL, LLC
635    SUMMIT GREENWICH URBAN RENEWAL, LLC
Name: owners_name, Length: 636, dtype: object